#Import the Dependencies

In [1]:
import numpy as np   
import pandas as pd
from joblib import dump

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#Data collection and processing

In [2]:
#loading the csv data to a Pandas Dataframe
heart_data = pd.read_csv('dataset.csv')

#print first 5 rows of dataframe
heart_data.head()


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [3]:
#checking the distribution of the target variable
heart_data['num'].value_counts()

num
0    411
1    265
2    109
3    107
4     28
Name: count, dtype: int64

In [4]:
#target data is imbalance
#Combine classes into a single 'has heart disease' class
heart_data['num'] = heart_data['num'].apply(lambda x: 1 if x > 0 else 0)

# Check the new distribution
print(heart_data['num'].value_counts())


num
1    509
0    411
Name: count, dtype: int64


In [5]:
#getting info of the dataset
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [6]:
#Statistical measures of the data
heart_data.describe()

,id,age,trestbps,chol,thalch,oldpeak,ca,num
count,920.000000,920.000000,861.000000,890.000000,865.000000,858.000000,309.000000,920.000000
mean,460.500000,53.510870,132.132404,199.130337,137.545665,0.878788,0.676375,0.553261
std,265.725422,9.424685,19.066070,110.780810,25.926276,1.091226,0.935653,0.497426
min,1.000000,28.000000,0.000000,0.000000,60.000000,-2.600000,0.000000,0.000000
25%,230.750000,47.000000,120.000000,175.000000,120.000000,0.000000,0.000000,0.000000
50%,460.500000,54.000000,130.000000,223.000000,140.000000,0.500000,0.000000,1.000000
75%,690.250000,60.000000,140.000000,268.000000,157.000000,1.500000,1.000000,1.000000
max,920.000000,77.000000,200.000000,603.000000,202.000000,6.200000,3.000000,1.000000


In [7]:
#seperate the features and target variables

X = heart_data.drop(columns=['id','num','dataset'])   #creating new dataframe 'X' from the original dataframe except for the 'num', 'id' and 'dataset' columns
y = heart_data['num']  

In [8]:
#Identify the categorical and numerical columns

categorical_cols = X.select_dtypes(include=['object']).columns  #string data types
numerical_cols = X.select_dtypes(include=['int64','float64']).columns  #float and int data types

print(f"Categorical : {categorical_cols} \n Numerical : {numerical_cols}")

Categorical : Index(['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'], dtype='object') 
 Numerical : Index(['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca'], dtype='object')


In [9]:
#Preprocessing pipeline for numerical data

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),   #replace the missing values with the mean value of that column
    ('scaler', StandardScaler())                   #scales the features to a mean of 0 and a standard deviation of 1.
])

In [10]:
#Preprocessing pipeline for categorical data

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # encode categorical data into numerical data
])

In [11]:
# Combine both transformers into a single ColumnTransformer

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [12]:
# Apply the transformation to the dataset

X_preprocessed = preprocessor.fit_transform(X)

In [13]:
# Extract the names of the numerical columns
numerical_columns = numerical_cols

# Extract the names of the categorical columns after one-hot encoding
categorical_columns = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols)

# Combine both numerical and categorical column names into one list
all_column_names = list(numerical_columns) + list(categorical_columns)

# Convert the preprocessed data into a DataFrame with the identified column names
df_preprocessed = pd.DataFrame(X_preprocessed, columns=all_column_names)

# Display the DataFrame
print(df_preprocessed.head())


        age  trestbps      chol    thalch   oldpeak        ca  sex_Female  \
0  1.007386  0.698041  0.311021  0.495698  1.349421 -1.249371         0.0   
1  1.432034  1.511761  0.797713 -1.175955  0.589832  4.292099         0.0   
2  1.432034 -0.658158  0.274289 -0.340128  1.634267  2.444942         0.0   
3 -1.752828 -0.115679  0.467130  1.968345  2.488805 -1.249371         0.0   
4 -1.328180 -0.115679  0.044717  1.371326  0.494884 -1.249371         1.0   

   sex_Male  cp_asymptomatic  cp_atypical angina  ...  restecg_normal  \
0       1.0              0.0                 0.0  ...             0.0   
1       1.0              1.0                 0.0  ...             0.0   
2       1.0              1.0                 0.0  ...             0.0   
3       1.0              0.0                 0.0  ...             1.0   
4       0.0              0.0                 1.0  ...             0.0   

   restecg_st-t abnormality  exang_False  exang_True  slope_downsloping  \
0                      

In [14]:
df_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       920 non-null    float64
 1   trestbps                  920 non-null    float64
 2   chol                      920 non-null    float64
 3   thalch                    920 non-null    float64
 4   oldpeak                   920 non-null    float64
 5   ca                        920 non-null    float64
 6   sex_Female                920 non-null    float64
 7   sex_Male                  920 non-null    float64
 8   cp_asymptomatic           920 non-null    float64
 9   cp_atypical angina        920 non-null    float64
 10  cp_non-anginal            920 non-null    float64
 11  cp_typical angina         920 non-null    float64
 12  fbs_False                 920 non-null    float64
 13  fbs_True                  920 non-null    float64
 14  restecg_lv

In [15]:
df_preprocessed.to_csv("preprocesed.csv")

In [16]:
# Split the preprocessed data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y,test_size=0.3, stratify=y, random_state=2)

In [17]:
print(X.shape, X_train.shape, X_test.shape)

(920, 13) (644, 25) (276, 25)


#Model Training

In [18]:
#Logistic Regression Model

model = LogisticRegression()

In [19]:
#training the logistic regression model with training data

model.fit(X_train, y_train)

LogisticRegression()

Model Evaluation

In [20]:
#Accuracy Score

#Accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [21]:
print("Accuracy on training data :",training_data_accuracy)

Accuracy on training data : 0.8385093167701864


In [22]:
#Accuracy on test data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [23]:
print("Accuracy on testing data :", testing_data_accuracy)

Accuracy on testing data : 0.7789855072463768


In [25]:
#Save the model
dump(model,'.//TrainedModels//LogisticRegressionModel.joblib')

['.//TrainedModels//LogisticRegressionModel.joblib']